# Run learners in job scripts

## Define the learners

We need the following variables:
* `learners` a list of learners
* `combos` a list of dicts of parameters that describe each learner
* `fnames` a list of filenames of each learner

In [ ]:
%%writefile _learners.py

import adaptive
from functools import partial

import funcs

syst_pars = dict(a=4, L=40, r=20, shape="square", dim=3)

params = dict(g=50, mu=100, B_y=0, B_x=0, **funcs.constants_InAs)

Ls = [1000, 2000, 3000, 5000, 10000]
l_Rs = [200] #np.geomspace(30, 1000, 10).tolist() + [np.inf]
l_es = [20, 50, 100, 200, 300, 500]
rs = [25]

combos = adaptive.utils.named_product(l_e=l_es, L=Ls, r=rs, l_R=l_Rs)

learners = []
fnames = []
folder = "data/q_phi_scaling_square_wire_new/"
for combo in combos:
    f = partial(
        funcs.conductance_1D,
        x_name='B_z',
        value_dict=combo,
        syst_pars=syst_pars,
        params=params,
    )
    learner = adaptive.AverageLearner1D(f, bounds=(0, 0.25))
    learner.average_priority = 1
    learner.min_seeds_per_point = 20
    fnames.append(f"{folder}_{combo}")
    learners.append(learner)

learner = adaptive.BalancingLearner(learners, strategy='cycle')

In [ ]:
# Execute the previous code block and plot the learners
from _learners import *
adaptive.notebook_extension()
learner.load(fnames)
learner.plot()

## The Python script that is being run in the job

In [ ]:
# Make sure to use the headnode's IP below.
import socket
import zmq.ssh
ip = socket.gethostbyname(socket.gethostname())
port = zmq.ssh.tunnel.select_random_ports(1)[0]
print(f'tcp://{ip}:{port}')

In [ ]:
%%writefile run_learner.py

import adaptive
from mpi4py.futures import MPIPoolExecutor

from adaptive.scheduler import client_support

url = "tcp://10.76.0.5:57681"

if __name__ == "__main__":
    learner, fname = client_support.get_learner(url)
    learner.load(fname)
    ex = MPIPoolExecutor()
    runner = adaptive.Runner(
        learner,
        executor=ex,
        goal=None,
        shutdown_executor=True,
        ioloop=None,
        retries=10,
        raise_if_retries_exceeded=False,
    )
    runner.start_periodic_saving(dict(fname=fname), interval=600)
    runner.ioloop.run_until_complete(runner.task)  # wait until runner goal reached
    client_support.is_done(url, fname)

# Import the files that were created

In [ ]:
import asyncio
from importlib import reload

from adaptive.scheduler import server_support
from pprint import pprint
from tinydb import TinyDB

_learners, run_learner

reload(_learners)
reload(run_learner)

db_fname = 'running.tinydb'

In [ ]:
# Create a new database
server_support.create_empty_db(db_fname, _learners.fnames, _learners.combos)

## Check the running learners
All the onces that are `None` are still `PENDING` or are not scheduled.

In [ ]:
with TinyDB(db_fname) as db:
    pprint(db.all())

## Start the job scripts

In [ ]:
# Get some unique names for the jobs
job_names = [f"WAL-{i}" for i in range(len(_learners.learners))]

ioloop = asyncio.get_event_loop()

database_task = ioloop.create_task(
    server_support.manage_database("tcp://*:57681", db_fname)
)

job_task = ioloop.create_task(
    server_support.manage_jobs(job_names, db_fname, ioloop, cores=50*8, interval=60)
)

In [ ]:
job_task.cancel(), database_task.cancel()

In [ ]:
job_task.print_stack()

In [ ]:
database_task.print_stack()